# Load Model from Huggingface Hub

To use a model with the Transformer Extension one needs to download it from the Huggingface Hub and put it into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm). Please refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about model loading functionality it provides.

There are two ways of doing this.

1. Using the TE_MODEL_DOWNLOADER_UDF UDF.
2. Downloading a model to a local drive and subsequently uploading it into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm) using CLI or an API.

The first method requires the database machine to have internet access. The second method provides a workaround if this is a problem. Another advantage of the second method is that it caches downloaded models on the local drive. This can make the model transfer quicker if it needs to be repeated.

<b>This notebook is not supposed to be run on its own. It contains model loading functions that are called by other notebooks.</b>

## Loading model with UDF

Here is the first way of loading the model. We wrap it into a function so that other notebooks can call it.

In [ ]:
def load_huggingface_model_udf(model_name: str) -> None:
    """
    Loads specified model into BucketFS using a UDF provided with the Transformer Extension.
    There must be a global variable - sb_config - providing access to the secret store holding
    the configuration data.

    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    """

    from exasol.connections import open_pyexasol_connection
    from exasol.language_container_activation import get_activation_sql
    from exasol.ai_lab_config import AILabConfig as CKey

    sql = f"""
    SELECT {sb_config.get(CKey.db_schema)}.TE_MODEL_DOWNLOADER_UDF(
        '{model_name}',
        '{sb_config.get(CKey.te_models_bfs_dir)}',
        '{sb_config.get(CKey.te_bfs_connection)}',
        '{sb_config.get(CKey.te_hf_connection)}'
    )
    """

    with open_pyexasol_connection(sb_config, compression=True) as conn:
        conn.execute(query=get_activation_sql(sb_config))
        conn.execute(query=sql)

## Loading model using the notebook

Here is the second way of loading the model. This method is very similar to the command line interface. For details on how to use the CLI please refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a>.

In [ ]:
def load_huggingface_model_cli(model_name: str, force_download: bool = False) -> None:
    """
    Loads specified model into BucketFS by saving it first to a local drive, as per the command-line interface.
    There must be a global variable - sb_config - providing access to the secret store holding
    the configuration data.

    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    force_download - If True the model will be reloaded from the hub even if it has been cached before.
    """

    from pathlib import Path
    import re
    from exasol.transformers_extension_wrapper import upload_model
    from exasol.ai_lab_config import AILabConfig as CKey

    # Make a name for the model sub-directory
    sub_dir = re.sub(r"[/\\?%*:|\"<>\x7F\x00-\x1F]", "-", model_name)
    models_dir = sb_config.get(CKey.te_models_cache_dir)
    cache_dir = str(Path(models_dir) / sub_dir)

    # Upload it to the BucketFS
    upload_model(sb_config, model_name=model_name, cache_dir=cache_dir, force_download=force_download)

## Method selector

This is the main entry point. The call will be dispatched to one of the above functions depending on the selected method.

In [ ]:
def load_huggingface_model(model_name: str, method: str = 'cli', force_download: bool = False) -> None:
    """
    Loads specified model into BucketFS choosing one of the two available methods.

    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    method         - The recognized values are "udf" and "cli". If the parameter is not set then
                     will look for method selection in the configuration. That failed the CLI method
                     is used. 
    force_download - If True the model will be reloaded from the hub even if it has been cached before.
    """

    method = method.lower()
    if method == 'udf':
        load_huggingface_model_udf(model_name)
    else:
        load_huggingface_model_cli(model_name, force_download=force_download)